Для воспроизводимости кода установим определенную версию tensorflow и keras

In [ ]:
!pip install -q tensorflow==2.1

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.backend import clear_session
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
print("We're using TF", tf.__version__)
print("We are using Keras", keras.__version__)

In [ ]:
# Проверяем что у нас работает GPU
tf.test.gpu_device_name()

In [ ]:
#GPU count and name
!nvidia-smi -L

In [ ]:
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

# MNIST

In [ ]:
# скачиваем данные
# (x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# что за данные
print(x_train.shape)
print(x_val.shape)
print (y_train[:10])
print (y_val[:10])

In [ ]:
# нарисуем картинку
x_train_image = np.tile(x_train[5, :, :].reshape((28, 28))[:, :, np.newaxis], (1, 1, 3))
print("Пример кусочка картинки (пиксели со значениями от 0 до 255):")
plt.imshow(x_train_image[15:20, 5:10], cmap="Greys")
plt.show()
print("Пример картинки:")
plt.imshow(x_train_image, cmap="Greys")
plt.show()
print("y_train [shape %s] 10 примеров:\n" % (str(y_train.shape)), y_train[:10])

In [ ]:
# Посмотрим побольше примеров
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap="Greys")
    plt.xlabel(y_train[i])
plt.show()

# Data

In [ ]:
# вытянем картинку в вектор размера 28*28=784 и будем использовать это как фичи!

x_train_float = x_train.reshape(-1, 28*28).astype(float)
print(x_train.shape, x_train_flat.shape)

x_val_float = x_val.reshape(-1, 28*28).astype(float)
print(x_val.shape, x_val_flat.shape)

In [ ]:
# # центрируем и нормируем, так сети будет проще учиться
# # x_train_float = x_train_flat.astype(np.float) / 255 - 0.5
# # x_val_float = x_val_flat.astype(np.float) / 255 - 0.5

# # по сути, это действия аналогичны нормализации через StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_flat = scaler.fit_transform(x_train_flat)
x_val_flat = scaler.transform(x_val_flat)

# # Но попробуйте пропустить нормирование и посмотрите что будет ;)
# x_train_float = x_train_flat.astype(np.float)
# x_val_float = x_val_flat.astype(np.float)

In [ ]:
# one-hot encode для ответов
y_train_oh = keras.utils.to_categorical(y_train, 10)
y_val_oh = keras.utils.to_categorical(y_val, 10)

print(y_train_oh.shape)
print(y_train_oh[:5], y_train[:5])

# Собираем модель

Самостоятельно постойте Multi layer Perceptron (MLP) классификатор (по аналогии с практическеской частью, но на новых данных)

In [ ]:
clear_session()

In [ ]:
# Тут ваш код Архитектуры сети
# не забудьте 10 выходов с активацией softmax на выходном слое!
# model = Sequential()
# model.add(Dense(..., input_shape=(28*28,))) # первый скрытый слой
# ...

model = Sequential()
model.add(Dense(units = 128, input_shape = (28 * 28,)))
model.add(Dense(units = 128))

model.add(Dense(units = 10, activation = 'softmax'))

In [ ]:
# model.summary() используется для вывода сводки (summary) архитектуры модели нейронной сети. 
# Эта сводка предоставляет информацию о количестве параметров модели и форме выхода каждого слоя.
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',  # минимизируем кросс-энтропию
    optimizer='adam',  # так будет быстрее, позже узнаем что это :)
    metrics=['accuracy']  # выводим процент правильных ответов
)

In [ ]:
history = model.fit(
                    x_train_float, 
                    y_train_oh,
                    batch_size=64,  # 64 объекта для подсчета градиента на каждом шаге
                    epochs=10,  # 10 проходов по датасету
                    validation_data=(x_val_float, y_val_oh)
                    )

In [ ]:
# Plot training & validation accuracy values
plt.style.use(['dark_background'])
fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.style.use(['dark_background'])
fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Получение предсказанных вероятностей
predicted_probs = model.predict(x_val_float)
# Преобразование вероятностей в индексы классов
predicted_classes = np.argmax(predicted_probs, axis=1)
# Вычисление точности
accuracy = accuracy_score(y_val, predicted_classes)
print('Test Accuracy:', accuracy)
